In [1]:
import math
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl
from __future__ import division
mpl.rc('figure', figsize=[10,6])

In [2]:
def updateFlag(val,sorted_attr_row):
    for i in range(len(sorted_attr_row)-1):
        val[i] = sorted_attr_row[i+1]

def defineT(attr):
    dtype = [('Id',int)]
    for i in range(len(attr)):
        dtype.append((attr[i],'S10'))
    return dtype

In [3]:
# Grouping by node attributes
def ACompatible(graph,edge,attr):
    dtype = defineT(attr)
    value = list(graph[attr].itertuples())
    original_table = np.asarray(value,dtype=dtype)
    groupArr = []

    sorted_attr = np.sort(original_table, order=attr)  
    val = ['','']
    for i in range(len(sorted_attr)):
        for j in range(len(attr)):
            if(sorted_attr[i][j+1]==val[j]):
                if(j == len(attr)-1):
                    newGroup.append(sorted_attr[i])
            else:
                newGroup = [(sorted_attr[i])]
                groupArr.append(newGroup)
                updateFlag(val,sorted_attr[i])
                break
                
    return groupArr

In [19]:
# Read graph data in csv files
vertex = pd.read_csv('proSumData/vertices_eg1.csv')
edge = pd.read_csv('proSumData/edges_eg1.csv')
attr = ['Type']
print vertex
print edge

  Vertex Type  Group
0     n1    E    NaN
1     n2    A    NaN
2     n3    E    NaN
3     n4    A    NaN
4     n5    E    NaN
5     n6    A    NaN
  Edge   n1   n2   n3   n4   n5  n6
0   n1  NaN  1.0  NaN  NaN  NaN NaN
1   n2  NaN  NaN  3.0  NaN  NaN NaN
2   n3  NaN  NaN  NaN  NaN  NaN NaN
3   n4  NaN  NaN  2.0  NaN  NaN NaN
4   n5  NaN  NaN  NaN  1.0  NaN NaN
5   n6  2.0  NaN  NaN  NaN  2.0 NaN


In [20]:
# get A-Compatibla grouping
ACompResult = ACompatible(vertex, edge, attr)
ACompResult

[[(1, 'A'), (3, 'A'), (5, 'A')], [(0, 'E'), (2, 'E'), (4, 'E')]]

In [21]:
# convert into arrays
vertex = np.asarray(vertex)
edge = np.asarray(edge)

In [8]:
# Update data structures
def DataStructure(tempResult, tempVertex, edge):
    print "update data structure"
    groupNum = len(tempResult)
    print "group number: ", groupNum
    nodeNum = edge.shape[0]
    
    # in-groups bitMap
    mapsize = (nodeNum,groupNum)
    bitMap = np.zeros(mapsize)
    
    for i in range(nodeNum):
        for g in range(groupNum):
            groupSet = tempResult[g]
            for j in range(len(groupSet)):
                currentNode = groupSet[j][0]
                # update group index of each node
                vertex[currentNode][2] = g
                # update number of nodes from in-groups
                if(not pd.isnull(edge[currentNode][i+1])):
                    bitMap[i][g] = bitMap[i][g] + 1
    
    return vertex, bitMap

In [22]:
updateVertex, initBitMap = DataStructure(ACompResult, vertex, edge)
print "updated vertex: "
print updateVertex
print "init bitmap: "
print initBitMap

update data structure
group number:  2
updated vertex: 
[['n1' 'E' 1]
 ['n2' 'A' 0]
 ['n3' 'E' 1]
 ['n4' 'A' 0]
 ['n5' 'E' 1]
 ['n6' 'A' 0]]
init bitmap: 
[[ 1.  0.]
 [ 0.  1.]
 [ 2.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  0.]]


In [12]:
# Check group of a node and Split into subgroups
def Split(node, tempResult, tempVertex, tempBitMap):
    groupNum = tempBitMap.shape[1]
    currentGroup = tempVertex[node][2]
    groupSet = tempResult[currentGroup]
    subGroup1 = []
    subGroup2 = []
    
    for i in range(len(groupSet)):
        currentNode = groupSet[i][0]
        for g in range(groupNum):
            if(tempBitMap[currentNode][g] != tempBitMap[node][g]):
                # a new group
                tempVertex[currentNode][2] = groupNum
        # split into two subgroups
        if(tempVertex[currentNode][2] == currentGroup):
            subGroup1.append((currentNode, tempVertex[currentNode][1]))
        elif(tempVertex[currentNode][2] == groupNum):
            subGroup2.append((currentNode, tempVertex[currentNode][1]))
        
    if(subGroup2): # split
        tempResult.remove(groupSet)
        tempResult.append(subGroup1)
        tempResult.append(subGroup2)
        return True, tempVertex, tempResult
    else: # already same in-groups
        return False, tempVertex, tempResult

In [23]:
# BFS queue: n6, n1, n5, n2, n4, n3
# n6
isSplit, splitVertex, splitResult = Split(5, ACompResult, updateVertex, initBitMap)
print isSplit
print splitVertex
print splitResult

True
[['n1' 'E' 1]
 ['n2' 'A' 2]
 ['n3' 'E' 1]
 ['n4' 'A' 2]
 ['n5' 'E' 1]
 ['n6' 'A' 0]]
[[(0, 'E'), (2, 'E'), (4, 'E')], [(5, 'A')], [(1, 'A'), (3, 'A')]]


In [24]:
# update data structures
newVertex, newBitMap = DataStructure(splitResult, splitVertex, edge)
print newVertex
print newBitMap

update data structure
group number:  3
[['n1' 'E' 0]
 ['n2' 'A' 2]
 ['n3' 'E' 0]
 ['n4' 'A' 2]
 ['n5' 'E' 0]
 ['n6' 'A' 1]]
[[ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  2.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  0.]]


In [25]:
# n1
isSplit, splitVertex, splitResult = Split(0, ACompResult, updateVertex, initBitMap)
print isSplit
print splitVertex
print splitResult

True
[['n1' 'E' 0]
 ['n2' 'A' 2]
 ['n3' 'E' 2]
 ['n4' 'A' 2]
 ['n5' 'E' 0]
 ['n6' 'A' 1]]
[[(5, 'A')], [(1, 'A'), (3, 'A')], [(0, 'E'), (4, 'E')], [(2, 'E')]]


In [26]:
# update data structures
newVertex, newBitMap = DataStructure(splitResult, splitVertex, edge)
print newVertex
print newBitMap

update data structure
group number:  4
[['n1' 'E' 2]
 ['n2' 'A' 1]
 ['n3' 'E' 3]
 ['n4' 'A' 1]
 ['n5' 'E' 2]
 ['n6' 'A' 0]]
[[ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  2.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


In [27]:
# n5
isSplit, splitVertex, splitResult = Split(4, ACompResult, updateVertex, initBitMap)
print isSplit
print splitVertex
print splitResult

False
[['n1' 'E' 2]
 ['n2' 'A' 1]
 ['n3' 'E' 3]
 ['n4' 'A' 1]
 ['n5' 'E' 2]
 ['n6' 'A' 0]]
[[(5, 'A')], [(1, 'A'), (3, 'A')], [(0, 'E'), (4, 'E')], [(2, 'E')]]


In [28]:
# n2
isSplit, splitVertex, splitResult = Split(1, ACompResult, updateVertex, initBitMap)
print isSplit
print splitVertex
print splitResult

False
[['n1' 'E' 2]
 ['n2' 'A' 1]
 ['n3' 'E' 3]
 ['n4' 'A' 1]
 ['n5' 'E' 2]
 ['n6' 'A' 0]]
[[(5, 'A')], [(1, 'A'), (3, 'A')], [(0, 'E'), (4, 'E')], [(2, 'E')]]


In [29]:
# n4
isSplit, splitVertex, splitResult = Split(3, ACompResult, updateVertex, initBitMap)
print isSplit
print splitVertex
print splitResult

False
[['n1' 'E' 2]
 ['n2' 'A' 1]
 ['n3' 'E' 3]
 ['n4' 'A' 1]
 ['n5' 'E' 2]
 ['n6' 'A' 0]]
[[(5, 'A')], [(1, 'A'), (3, 'A')], [(0, 'E'), (4, 'E')], [(2, 'E')]]


In [30]:
# n3
isSplit, splitVertex, splitResult = Split(2, ACompResult, updateVertex, initBitMap)
print isSplit
print splitVertex
print splitResult

False
[['n1' 'E' 2]
 ['n2' 'A' 1]
 ['n3' 'E' 3]
 ['n4' 'A' 1]
 ['n5' 'E' 2]
 ['n6' 'A' 0]]
[[(5, 'A')], [(1, 'A'), (3, 'A')], [(0, 'E'), (4, 'E')], [(2, 'E')]]


In [ ]:
# Graph summarization algorithm
def IPS(src, vertex, edge, attr):